In [1]:
import pandas as pd
import json
import glob
import os

In [2]:
# settings

folder_path = "../data/19th_20th_speeches_all"
test_file = "LP_20_Sitzung_105.json"

#### **Get files and file content**

In [3]:
# get a list of all filenames
json_files = glob.glob(os.path.join(folder_path, "*.json"))

print("Nr. of session protocols:", len(json_files))
json_files[:5]

Nr. of session protocols: 438


['../data/19th_20th_speeches_all\\LP_19_Sitzung_1.json',
 '../data/19th_20th_speeches_all\\LP_19_Sitzung_10.json',
 '../data/19th_20th_speeches_all\\LP_19_Sitzung_100.json',
 '../data/19th_20th_speeches_all\\LP_19_Sitzung_101.json',
 '../data/19th_20th_speeches_all\\LP_19_Sitzung_102.json']

In [4]:
# get a test file (later in loop over all json files)
with open(f"{folder_path}/{test_file}", 'r') as file:
    data = json.load(file)


# print the data
print(data.keys())

dict_keys(['Protocol', 'AgendaItems', 'NLPSpeeches', 'SpeechCategories'])


***
## **Get Agenda Item and Protocol Info**
***

### **Protocol data**

In [6]:
data['Protocol']

{'Date': '2023-05-24T00:00:00',
 'LegislaturePeriod': 20,
 'Number': 105,
 'Title': '105.\xa0Sitzung',
 'AgendaItemsCount': 8,
 'MongoId': '00000000-0000-0000-0000-000000000000',
 'Id': '211ba62b-b557-4b0d-e458-08dbb6b04007'}

In [7]:
rows = []

for session_file in json_files:
    with open(session_file, 'r') as file:
        data_temp = json.load(file)
        rows.append(data_temp['Protocol'])

# build final DataFrame in one step
df_protocol_meta = pd.DataFrame(rows)[['Date', 'LegislaturePeriod', 'Number', 'Title', 'AgendaItemsCount', 'MongoId', 'Id']]

In [8]:
df_protocol_meta.rename(columns={
    'Date': 'ProtocolDate',
    'Title': 'SessionTitle'
}, inplace=True)

In [9]:
df_protocol_meta['DateOnly'] = pd.to_datetime(df_protocol_meta['ProtocolDate']).dt.date

In [10]:
df_protocol_meta

,ProtocolDate,LegislaturePeriod,Number,SessionTitle,AgendaItemsCount,MongoId,Id,DateOnly
0,2017-10-24T00:00:00,19,1,1. Sitzung,6,6205403f03957565211832cb,76c3b183-81d7-4cd0-ee48-08da0c32b602,2017-10-24
1,2018-01-31T00:00:00,19,10,10. Sitzung,3,6205403f03957565211832c2,9830dca7-f0e1-4906-ee3f-08da0c32b602,2018-01-31
2,2019-05-15T00:00:00,19,100,100. Sitzung,4,6205403f0395756521183268,7d5aebcf-9b16-47a6-ede5-08da0c32b602,2019-05-15
3,2019-05-16T00:00:00,19,101,101. Sitzung,26,6205403f0395756521183267,f98bb208-7f24-4a3f-ede4-08da0c32b602,2019-05-16
4,2019-05-17T00:00:00,19,102,102. Sitzung,11,6205403f0395756521183266,ee086175-6c20-44b0-ede3-08da0c32b602,2019-05-17
...,...,...,...,...,...,...,...,...
433,2023-03-31T00:00:00,20,95,95. Sitzung,7,00000000-0000-0000-0000-000000000000,218f1cf4-f452-45ff-5c0a-08db39b0f643,2023-03-31
434,2023-04-19T00:00:00,20,96,96. Sitzung,7,00000000-0000-0000-0000-000000000000,797bcf60-4df4-462a-9463-08db4195fa79,2023-04-19
435,2023-04-20T00:00:00,20,97,97. Sitzung,14,00000000-0000-0000-0000-000000000000,0f039fcd-49f1-4c25-159d-08db424421b3,2023-04-20
436,2023-04-21T00:00:00,20,98,98. Sitzung,6,00000000-0000-0000-0000-000000000000,281683d2-1125-46ff-d5ef-08db44e7eb96,2023-04-21


In [11]:
# sanity checks

print("Nr. of sessions in 19th: ", len(df_protocol_meta[df_protocol_meta['LegislaturePeriod'] == 19])) # fine
print("Nr. of sessions in 20th: ", len(df_protocol_meta[df_protocol_meta['LegislaturePeriod'] == 20])) # fine


Nr. of sessions in 19th:  239
Nr. of sessions in 20th:  199


### **Agenda Item Meta Data**

In [12]:
data['AgendaItems'][0]

{'Title': 'Pflegeunterstützungs- und -entlastungsgesetz',
 'Description': 'Erste Beratung des von der Bundesregierung eingebrachten Entwurfs eines Gesetzes zur Unterstützung und Entlastung in der Pflege (Pflegeunterstützungs- und -entlastungsgesetz – PUEG)<br>Drucksache <a target="_blank" href="https://dserver.bundestag.de/btd/20/068/2006869.pdf" class="dipLink" title="Die PDF-Datei Drucksache 20/6869 öffnet sich in einem neuen Fenster"><i class="icon-doc"></i>20/6869</a>',
 'AgendaItemNumber': '1',
 'Order': 1,
 'Date': '2023-05-24T13:00:00',
 'ProtocolId': '211ba62b-b557-4b0d-e458-08dbb6b04007',
 'Id': '13e81aed-136d-4d56-a78b-08dbb6be6ecc'}

In [13]:
rows = []

for session_file in json_files:
    with open(session_file, 'r') as file:
        data_temp = json.load(file)

    for item in data_temp["AgendaItems"]:
        rows.append({
            'AgendaItemTitle': item['Title'],
            'AgendaItemNumber': item['AgendaItemNumber'],
            'Order': item['Order'],
            'AgendaItemDate': item['Date'],
            'ProtocolId': item['ProtocolId']
        })

df_agenda_items = pd.DataFrame(rows)


# ~30 sec. runtime

In [14]:
df_agenda_items['DateOnly'] = pd.to_datetime(df_agenda_items['AgendaItemDate']).dt.date


In [15]:
df_agenda_items

,AgendaItemTitle,AgendaItemNumber,Order,AgendaItemDate,ProtocolId,DateOnly
0,Eröffnung der Sitzung durch den Alterspräsidenten,1,1,2017-10-24T11:05:00,76c3b183-81d7-4cd0-ee48-08da0c32b602,2017-10-24
1,Beschlussfassung über die Geschäftsordnung,2,2,2017-10-24T11:25:00,76c3b183-81d7-4cd0-ee48-08da0c32b602,2017-10-24
2,Wahl des Präsidenten,3,3,2017-10-24T12:05:00,76c3b183-81d7-4cd0-ee48-08da0c32b602,2017-10-24
3,Amtsübernahme durch den Präsidenten mit Ansprache,4,4,2017-10-24T13:15:00,76c3b183-81d7-4cd0-ee48-08da0c32b602,2017-10-24
4,Festlegung der Zahl der Stellvertreter des Prä...,5,5,2017-10-24T13:40:00,76c3b183-81d7-4cd0-ee48-08da0c32b602,2017-10-24
...,...,...,...,...,...,...
4459,Aktuelle Stunde - Umstrittene Personalpolitik ...,ZP 1,3,2023-04-26T14:40:00,7b867a5b-45a6-438f-ae05-08db4746451b,2023-04-26
4460,Bundeswehreinsatz Evakuierung aus Sudan,ZP 2,4,2023-04-26T15:30:00,7b867a5b-45a6-438f-ae05-08db4746451b,2023-04-26
4461,Wolfsbestandsmanagement,4,5,2023-04-26T16:40:00,7b867a5b-45a6-438f-ae05-08db4746451b,2023-04-26
4462,Bundeswehreinsatz EUNAVFOR MED IRINI,3,6,2023-04-26T17:30:00,7b867a5b-45a6-438f-ae05-08db4746451b,2023-04-26


***
### **Merge**
***

In [16]:
df_agenda = df_agenda_items.merge(df_protocol_meta, on='DateOnly', how='left')

In [17]:
df_agenda.to_csv("../data/agenda_item_info.csv")